In [7]:
import matplotlib.animation as animation
import pandas as pd
import numpy as np
from copy import deepcopy
from itertools import chain
import random
import matplotlib.pyplot as plt
import math
import folium
from matplotlib import colormaps as cm
from scipy import stats
import time
import os
from selenium import webdriver
from tqdm import tqdm
from selenium.webdriver.firefox.options import Options
import imageio
from PIL import Image, ImageDraw, ImageFont

from pymoo.indicators.hv import Hypervolume
from pymoo.indicators.igd import IGD

In [8]:
file_path1 = '../data/distance.xlsx'
file_path2 = '../data/capacity.xlsx'
file_path3 = '../data/population.xlsx'

df_distance = pd.read_excel(file_path1)
df_capacity = pd.read_excel(file_path2)
df_population = pd.read_excel(file_path3)
df_walking = df_distance.copy()  # Start with a copy of the original
df_driving = df_distance.copy()

for column in df_distance.columns:
    if column.startswith('sh'):
        # Split the column
        split_result = df_distance[column].str.split('/', expand=True)
        # Assign values to new DataFrames
        df_walking[column] = split_result[0]  # Walking distances
        if 1 in split_result.columns:
            df_driving[column] = split_result[1]  # Driving distances
        else:
            df_driving[column] = None  # Handle missing driving distances

building_blocks = {}
shelters = {}
for index, row in df_capacity.iterrows():
    shelter_name = row['Shelter']
    capacity = int(row['Capacity'])
    lat_lng = row['Lat_Lng']
    lat, lng = map(float, row['Lat_Lng'].split(', '))
    shelters[index] = {'capacity': capacity, 'lat': lat, 'lng': lng}
for index, row in df_population.iterrows():
    block_name = row['Building_Block']
    population = int(row['Population'])
    lat, lng = map(float, row['Lat_Lng'].split(', '))
    building_blocks[index] = {'population': population, 'lat': lat, 'lng': lng}
print(len(shelters))
print(len(building_blocks))    

42
44


visualization on map

In [9]:
def get_color(value, min_value, max_value):
  """
  Maps a value to a color in a green-to-red colormap.
  """
  if value > max_value:
    return '#{:02x}{:02x}{:02x}'.format(0, 0, 0)
  norm_value = (value - min_value) / (max_value - min_value)
  cmap = cm.get_cmap('RdYlGn_r')  # Reverse green-yellow-red colormap
  color = cmap(norm_value)[:3]  # Convert to RGB tuple
  return '#{:02x}{:02x}{:02x}'.format(int(color[0] * 255), int(color[1] * 255), int(color[2] * 255))

def map_animation(individual, filename):
        number_of_shelters = len(shelters)
        number_of_blocks = len(building_blocks)
        shelter_overload = [0.0 for _ in range(number_of_shelters)]
        m = folium.Map(location=[-25.8900, 32.6158], zoom_start=11.5)
        for i in range (len(individual)):
            building_block = building_blocks[i]
            shelter = shelters[individual[i] - 1]
            shelter_overload[individual[i] - 1] += float(df_population.iloc[i,1])
            point_1 = [building_block['lat'], building_block['lng']]
            point_2 = [shelter['lat'], shelter['lng']]
            folium.PolyLine([point_1, point_2], color="black", weight=1).add_to(m)
        
        color = []
        icon_props = dict(prefix='fa', icon='user', color='black')
        for i in range (number_of_shelters):
            cap = df_capacity.iloc[i,1]
            shelter_overload[i] = shelter_overload[i] - float(cap)
            overload = shelter_overload[i]
            shelter = shelters[i]
            max_value = float(cap) * 3/2
            min_value = -1 * float(cap)
            color = get_color(overload, min_value, max_value)
            text = str(overload)
            folium.CircleMarker(
            location = [shelter['lat'], shelter['lng']],
            radius =  shelter['capacity'] / 50,  # radius in pixels
            color = color,
            fill = True,
            fill_color = color,
            fill_opacity = 1,
            tooltip = f"{overload}",
            icon=folium.Icon(**icon_props, text=text, icon_size=(2, 2))
            ).add_to(m)
        for i in range(number_of_blocks):
            building_block = building_blocks[i]
            folium.CircleMarker(
            location = [building_block['lat'], building_block['lng']],
            radius = building_block['population'] / 50,  # radius in pixels
            color = 'blue',
            fill = True,
            fill_color ='blue',
            fill_opacity = 1
            ).add_to(m)
        # Save the map as an image
        m.save(filename)
        return filename

def convert_html_to_png(path, pare_to_number,):
    options = Options()
    # for i in range(numb_iteration): 
    filename = path
    mapURL = 'file://{0}/{1}'.format(os.getcwd(),filename)
    driver = webdriver.Firefox(options=options)
    driver.get(mapURL)
    time.sleep(1)
    filename2 = f'map_animation_{pare_to_number}.png'
    driver.save_screenshot(filename2)
    driver.quit()
    write_on_picture(pare_to_number,filename2)

def textsize(text, font):
    im = Image.new(mode="P", size=(0, 0))
    draw = ImageDraw.Draw(im)
    _, _, width, height = draw.textbbox((0, 0), text=text, font=font)
    return width, height

def write_on_picture(pare_to_number,filename2):
    font_path = "Candara.ttf"
    font_size = 60
    text_color = (0, 0, 0)
    font = ImageFont.truetype(font_path, font_size)
    image_path = filename2
    img = Image.open(image_path)
    draw = ImageDraw.Draw(img)
    text = f" number in front : {pare_to_number}"
    text_width, text_height = textsize(text, font=font)
    x = 500  # Adjust horizontal position
    y = 80  # Adjust vertical position
    draw.text((x, y), text, fill=text_color, font=font)
    # Save the modified image
    img.save(filename2)

def make_animation(Front):
    filenames = []
    for i in range(len(Front)):
            filenames.append(f"map_animation_{i}.png")
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
        print(f"Image {filename} shape: {imageio.imread(filename).shape}")
    # Create a GIF animation (adjust duration as needed)
    imageio.mimsave('map_animation.gif', images, fps = 3)  
 
def make_animation2():
    filenames = []
    for i in range (500):
        filenames.append(f"pareToFront_of_generation_{i}.png")   
    images = []
    for filename in filenames:
        images.append(imageio.imread(filename))
        print(f"Image {filename} shape: {imageio.imread(filename).shape}")
    # Create a GIF animation (adjust duration as needed)
    imageio.mimsave('pare_to_animation.gif', images, fps = 9) 

NSGA-II implementation :

source code from : https://github.com/baopng/NSGA-II

In [10]:

class Individual(object):

    def __init__(self):
        self.rank = None
        self.crowding_distance = None
        self.domination_count = None
        self.dominated_solutions = None
        self.features = None
        self.objectives = None

    def __eq__(self, other):
        if isinstance(self, other.__class__):
            return self.features == other.features
        return False

    def dominates(self, other_individual):
        and_condition = True
        or_condition = False
        for first, second in zip(self.objectives, other_individual.objectives):
            and_condition = and_condition and first <= second
            or_condition = or_condition or first < second
        return (and_condition and or_condition)
    

class Population:

    def __init__(self):
        self.population = []
        self.fronts = []

    def __len__(self):
        return len(self.population)

    def __iter__(self):
        return self.population.__iter__()

    def extend(self, new_individuals):
        self.population.extend(new_individuals)

    def append(self, new_individual):
        self.population.append(new_individual)



class Problem:

    def __init__(self, objectives, num_of_variables, variables_range):
        self.num_of_objectives = len(objectives)
        self.num_of_variables = num_of_variables
        self.objectives = objectives
        self.variables_range = variables_range

    def generate_individual(self):
        individual = Individual()
        individual.features = [random.randint(1,self.variables_range) for _ in range(self.num_of_variables)]
        return individual

    def calculate_objectives(self, individual):
        individual.objectives = [f(individual.features) for f in self.objectives]


class NSGA2Utils:

    def __init__(self, problem, num_of_individuals=100,
                 num_of_tour_particips=2, tournament_prob=0.9, mutation_param=0.01):

        self.problem = problem
        self.num_of_individuals = num_of_individuals
        self.num_of_tour_particips = num_of_tour_particips
        self.tournament_prob = tournament_prob
        self.mutation_param = mutation_param

    def create_initial_population(self):
        population = Population()
        for _ in range(self.num_of_individuals):
            individual = self.problem.generate_individual()
            self.problem.calculate_objectives(individual)
            population.append(individual)
        return population

    def fast_nondominated_sort(self, population):
        population.fronts = [[]]
        for individual in population:
            individual.domination_count = 0
            individual.dominated_solutions = []
            for other_individual in population:
                if individual.dominates(other_individual):
                    individual.dominated_solutions.append(other_individual)
                elif other_individual.dominates(individual):
                    individual.domination_count += 1
            if individual.domination_count == 0:
                individual.rank = 0
                population.fronts[0].append(individual)
        i = 0
        while len(population.fronts[i]) > 0:
            temp = []
            for individual in population.fronts[i]:
                for other_individual in individual.dominated_solutions:
                    other_individual.domination_count -= 1
                    if other_individual.domination_count == 0:
                        other_individual.rank = i + 1
                        temp.append(other_individual)
            i = i + 1
            population.fronts.append(temp)

    def calculate_crowding_distance(self, front):
        if len(front) > 0:
            solutions_num = len(front)
            for individual in front:
                individual.crowding_distance = 0

            for m in range(len(front[0].objectives)):
                front.sort(key=lambda individual: individual.objectives[m])
                front[0].crowding_distance = 10 ** 9
                front[solutions_num - 1].crowding_distance = 10 ** 9
                m_values = [individual.objectives[m] for individual in front]
                scale = max(m_values) - min(m_values)
                if scale == 0: scale = 1
                for i in range(1, solutions_num - 1):
                    front[i].crowding_distance += (front[i + 1].objectives[m] - front[i - 1].objectives[m]) / scale

    def crowding_operator(self, individual, other_individual):
        if (individual.rank < other_individual.rank) or \
                ((individual.rank == other_individual.rank) and (
                        individual.crowding_distance > other_individual.crowding_distance)):
            return 1
        else:
            return -1

    def create_children(self, population):
        children = []
        while len(children) < len(population):
            parent1 = self.__tournament(population)
            parent2 = parent1
            while parent1 == parent2:
                parent2 = self.__tournament(population)
            child1, child2 = self.__crossover(parent1, parent2)
            if self.__choose_with_prob(self.mutation_param):
                child1 = self.__mutate(child1)
            if self.__choose_with_prob(self.mutation_param):
                child2 = self.__mutate(child2)
            self.problem.calculate_objectives(child1)
            self.problem.calculate_objectives(child2)
            children.append(child1)
            children.append(child2)

        return children

    def __crossover(self, parent1, parent2):
        child1 = self.problem.generate_individual()
        child2 = self.problem.generate_individual()
        crossover_point1 = random.randint(0, len(parent1.features) - 2)
        crossover_point2 = random.randint(crossover_point1 + 1, len(parent1.features) - 1)
        child1.features = parent1.features[:crossover_point1] + parent2.features[crossover_point1:crossover_point2] + parent1.features[crossover_point2:]
        child2.features = parent2.features[:crossover_point1] + parent1.features[crossover_point1:crossover_point2] + parent2.features[crossover_point2:]
        return child1, child2

    def __mutate(self, child):
        mutation_points1 = random.randint(0,self.problem.num_of_variables - 1)
        mutation_points2 = random.randint(0,self.problem.num_of_variables - 1)
        while mutation_points1 == mutation_points2:
            mutation_points2 = random.randint(0,self.problem.num_of_variables - 1)
        child.features[mutation_points1], child.features[mutation_points2] = child.features[mutation_points2], child.features[mutation_points1]
        return child

    def __tournament(self, population):
        participants = random.sample(population.population, self.num_of_tour_particips)
        best = None
        for participant in participants:
            if best is None or (
                    self.crowding_operator(participant, best) == 1 and self.__choose_with_prob(self.tournament_prob)):
                best = participant
        return best

    def __choose_with_prob(self, prob):
        if random.random() <= prob:
            return True
        return False


class Evolution:

    def __init__(self, problem, num_of_generations=500, num_of_individuals=100, num_of_tour_particips=2,
                 tournament_prob=0.9, mutation_param=0.01, use_threshold_flag = True):
        self.utils = NSGA2Utils(problem, num_of_individuals, num_of_tour_particips, tournament_prob,
                                mutation_param)
        self.use_threshold_flag = use_threshold_flag
        self.population = None
        self.num_of_generations = num_of_generations
        self.num_of_individuals = num_of_individuals
 
    def threshold_calculator(self, now_pare_to_front, last_pare_to_front):
        Fcapacity_Pare_To_now = []
        Fdistance_Pare_To_now = []
        Fcapacity_Pare_To_last = []
        Fdistance_Pare_To_last = []
        Pare_to_now = [[] for _ in range(len(now_pare_to_front))]
        Pare_to_last = [[] for _ in range(len(last_pare_to_front))]
        for j in now_pare_to_front:
            Fcapacity_Pare_To_now.append(j.objectives[1])
            Fdistance_Pare_To_now.append(j.objectives[0])
        for j in last_pare_to_front:
            Fcapacity_Pare_To_last.append(j.objectives[1])
            Fdistance_Pare_To_last.append(j.objectives[0])    
        Fdistance_max_last = max(Fdistance_Pare_To_last)
        Fdistance_min_last = min(Fdistance_Pare_To_last)
        Fcapacity_max_last = max(Fcapacity_Pare_To_last)
        Fcapacity_min_last = min(Fcapacity_Pare_To_last)
        Fdistance_max_now = max(Fdistance_Pare_To_now)
        Fdistance_min_now = min(Fdistance_Pare_To_now)
        Fcapacity_max_now = max(Fcapacity_Pare_To_now)
        Fcapacity_min_now = min(Fcapacity_Pare_To_now)
        difference_max = max(((Fdistance_max_last - Fdistance_max_now) / (Fdistance_max_now - Fdistance_min_now)), ((Fcapacity_max_last - Fcapacity_max_now) / (Fcapacity_max_now - Fcapacity_min_now))) 
        differnce_min = max(((Fdistance_min_last - Fdistance_min_now) / (Fdistance_max_now - Fdistance_min_now)), ((Fcapacity_min_last - Fcapacity_min_now) / (Fcapacity_max_now - Fcapacity_min_now))) 
        for i in range (len(Fcapacity_Pare_To_now)):
            Fdistance_Pare_To_now[i] = (Fdistance_Pare_To_now[i] - Fdistance_min_now) / (Fdistance_max_now - Fdistance_min_now)
            Fcapacity_Pare_To_now[i] = (Fcapacity_Pare_To_now[i] - Fcapacity_min_now) / (Fcapacity_max_now - Fcapacity_min_now)
            Pare_to_now[i] = [Fdistance_Pare_To_now[i], Fcapacity_Pare_To_now[i]]
        for i in range (len(Fcapacity_Pare_To_last)):
            Fdistance_Pare_To_last[i] = (Fdistance_Pare_To_last[i] - Fdistance_min_now) / (Fdistance_max_now - Fdistance_min_now)
            Fcapacity_Pare_To_last[i] = (Fcapacity_Pare_To_last[i] - Fcapacity_min_now) / (Fcapacity_max_now - Fcapacity_min_now)
            Pare_to_last[i] = [Fdistance_Pare_To_last[i], Fcapacity_Pare_To_last[i]]
        Pare_to_now = np.array(Pare_to_now)
        Pare_to_last = np.array(Pare_to_last)
        ind = IGD(Pare_to_now)
        igd = ind(Pare_to_last)
        return difference_max, differnce_min, igd
 
    
    def evolve(self):
        self.population = self.utils.create_initial_population()
        self.utils.fast_nondominated_sort(self.population)
        for front in self.population.fronts:
            self.utils.calculate_crowding_distance(front)
        children = self.utils.create_children(self.population)
        pare_to_front_of_last_generation = []
        pare_to_front_of_check_threshold = []
        start_threshold = 100
        window_size = 30
        threshold_flag = False
        finished_generation_number = 0
        # number_of_all_fronts = 0
        # returned_population = None
        for i in tqdm(range(self.num_of_generations)):
            self.population.extend(children)
            self.utils.fast_nondominated_sort(self.population)
            new_population = Population()
            front_num = 0
            while len(new_population) + len(self.population.fronts[front_num]) <= self.num_of_individuals:
                self.utils.calculate_crowding_distance(self.population.fronts[front_num])
                new_population.extend(self.population.fronts[front_num])
                front_num += 1
            self.utils.calculate_crowding_distance(self.population.fronts[front_num])
            self.population.fronts[front_num].sort(key=lambda individual: individual.crowding_distance, reverse=True)
            new_population.extend(self.population.fronts[front_num][0:self.num_of_individuals - len(new_population)])
            self.population = new_population
            self.utils.fast_nondominated_sort(self.population)
            for front in self.population.fronts:
                 self.utils.calculate_crowding_distance(front)
            pare_to_front_of_last_generation = self.population.fronts[0]
            finished_generation_number = i
            if(self.use_threshold_flag):
                if i > start_threshold :
                    pare_to_front_of_check_threshold.append(self.population.fronts[0])
                if i == start_threshold + window_size :  
                    differences_max = []
                    differnces_min = []
                    igds = []
                    for i in range(1,window_size):
                        difference_max, differnce_min, igd = self.threshold_calculator(now_pare_to_front  = pare_to_front_of_check_threshold[i], last_pare_to_front = pare_to_front_of_check_threshold[i - 1])
                        differences_max.append(difference_max)
                        differnces_min.append(differnce_min)
                        igds.append(igd)
                    if ((max(differences_max) < 0.03) and (max(differnces_min) < 0.01) and (max(igds) < 0.001)) :
                            threshold_flag = True
                    pare_to_front_of_check_threshold = []
                    start_threshold += window_size
                if threshold_flag :
                    break 
            children = self.utils.create_children(self.population)
        return  pare_to_front_of_last_generation, finished_generation_number

In [11]:
number_of_shelters = 42
number_of_blocks = 44

Objective functions

In [12]:
def fdistance(x):
  y = 0.0
  for i in range (len(x)):
    y += float(df_population.iloc[i,1]) * float(df_walking.iloc[i,x[i]])
  return y

def fcapacity(x):
  y = 0.0
  shelter_population = [0.0 for _ in range (number_of_shelters)]
  for i in range (len(x)):
    shelter_population[x[i] - 1] += int(df_population.iloc[i,1])
  for i in range (number_of_shelters):
    y += abs((shelter_population[i] / int(df_capacity.iloc[i,1])) - 1)
  return y

In [8]:
best_pare_to_front =  np.array([[1.00000000e+00, 0.00000000e+00]
 ,[1.00000000e+00, 0.00000000e+00]
 ,[9.41619203e-01, 8.38100027e-04]
 ,[9.41619203e-01, 8.38100027e-04]
 ,[8.93816111e-01, 1.67620005e-03]
 ,[8.93816111e-01, 1.67620005e-03]
 ,[8.76525631e-01, 2.37810883e-03]
 ,[8.76525631e-01, 2.37810883e-03]
 ,[8.49369406e-01, 2.77620634e-03]
 ,[8.49369406e-01, 2.77620634e-03]
 ,[8.33706265e-01, 3.21620885e-03]
 ,[8.31773800e-01, 4.05430888e-03]
 ,[7.95463792e-01, 4.63050265e-03]
 ,[7.88649308e-01, 5.73050894e-03]
 ,[7.62917006e-01, 6.24384520e-03]
 ,[7.58848657e-01, 9.65910281e-03]
 ,[7.32404394e-01, 1.04343453e-02]
 ,[7.28336046e-01, 1.38496030e-02]
 ,[7.11452400e-01, 1.42896055e-02]
 ,[7.04332791e-01, 1.51277055e-02]
 ,[6.97416599e-01, 1.64477130e-02]
 ,[6.80939788e-01, 1.84801056e-02]
 ,[6.80939788e-01, 1.84801056e-02]
 ,[6.67717657e-01, 2.01563057e-02]
 ,[6.52868186e-01, 2.26706057e-02]
 ,[6.42290480e-01, 2.43468058e-02]
 ,[6.36391375e-01, 2.65049134e-02]
 ,[6.18694060e-01, 2.76992059e-02]
 ,[6.05878763e-01, 3.06954135e-02]
 ,[5.96318145e-01, 3.18897060e-02]
 ,[5.76484947e-01, 3.61430637e-02]
 ,[5.69670464e-01, 3.84687913e-02]
 ,[5.55126119e-01, 3.92230813e-02]
 ,[5.55126119e-01, 3.92230813e-02]
 ,[5.35496338e-01, 4.34973914e-02]
 ,[5.25732303e-01, 4.84840866e-02]
 ,[5.06204231e-01, 4.85259916e-02]
 ,[4.97660700e-01, 5.26745867e-02]
 ,[4.81794142e-01, 5.35545917e-02]
 ,[4.76301871e-01, 5.57546043e-02]
 ,[4.68368592e-01, 6.17260670e-02]
 ,[4.56163548e-01, 6.26060720e-02]
 ,[4.50874695e-01, 6.29413120e-02]
 ,[4.40703824e-01, 7.02746873e-02]
 ,[4.36838893e-01, 7.21394598e-02]
 ,[4.21379170e-01, 7.38575649e-02]
 ,[4.08970708e-01, 7.77128250e-02]
 ,[4.05919447e-01, 8.15261801e-02]
 ,[3.90459723e-01, 8.32442852e-02]
 ,[3.90459723e-01, 8.32442852e-02]
 ,[3.76118796e-01, 9.42024431e-02]
 ,[3.76118796e-01, 9.42024431e-02]
 ,[3.68999186e-01, 9.93358057e-02]
 ,[3.52929211e-01, 1.03253923e-01]
 ,[3.52929211e-01, 1.03253923e-01]
 ,[3.35740439e-01, 1.10692061e-01]
 ,[3.35740439e-01, 1.10692061e-01]
 ,[3.14890155e-01, 1.18067341e-01]
 ,[3.06956876e-01, 1.24855952e-01]
 ,[2.90480065e-01, 1.29381692e-01]
 ,[2.82343369e-01, 1.35248392e-01]
 ,[2.82343369e-01, 1.35248392e-01]
 ,[2.73189585e-01, 1.45808452e-01]
 ,[2.65052889e-01, 1.50501812e-01]
 ,[2.65052889e-01, 1.50501812e-01]
 ,[2.49796583e-01, 1.61501875e-01]
 ,[2.49796583e-01, 1.61501875e-01]
 ,[2.40642799e-01, 1.72061936e-01]
 ,[2.40642799e-01, 1.72061936e-01]
 ,[2.34336859e-01, 1.81071511e-01]
 ,[2.34336859e-01, 1.81071511e-01]
 ,[2.21521562e-01, 1.98273514e-01]
 ,[2.18063466e-01, 2.00347812e-01]
 ,[2.12367779e-01, 2.08833574e-01]
 ,[2.12367779e-01, 2.08833574e-01]
 ,[2.04027665e-01, 2.20147925e-01]
 ,[2.04027665e-01, 2.20147925e-01]
 ,[1.92432872e-01, 2.33725145e-01]
 ,[1.92432872e-01, 2.33725145e-01]
 ,[1.89381611e-01, 2.48245228e-01]
 ,[1.78397071e-01, 2.53525258e-01]
 ,[1.74125305e-01, 2.66725334e-01]
 ,[1.69243287e-01, 2.67668196e-01]
 ,[1.64157852e-01, 2.79925409e-01]
 ,[1.64157852e-01, 2.79925409e-01]
 ,[1.58665582e-01, 2.94801685e-01]
 ,[1.50528885e-01, 3.00668385e-01]
 ,[1.47477624e-01, 3.15188468e-01]
 ,[1.40968267e-01, 3.21599933e-01]
 ,[1.36289666e-01, 3.27068536e-01]
 ,[1.32017901e-01, 3.40268611e-01]
 ,[1.32017901e-01, 3.40268611e-01]
 ,[1.22253865e-01, 3.54600122e-01]
 ,[1.22253865e-01, 3.54600122e-01]
 ,[1.20016273e-01, 3.66857334e-01]
 ,[1.11065907e-01, 3.78360257e-01]
 ,[1.10455655e-01, 3.87788883e-01]
 ,[1.05980472e-01, 3.94388920e-01]
 ,[1.03234337e-01, 4.06080416e-01]
 ,[1.01403580e-01, 4.19029061e-01]
 ,[9.27583401e-02, 4.27200536e-01]
 ,[8.93002441e-02, 4.33360572e-01]
 ,[8.29943043e-02, 4.50960672e-01]
 ,[8.29943043e-02, 4.50960672e-01]
 ,[8.14686737e-02, 4.61080730e-01]
 ,[7.94344996e-02, 4.65480755e-01]
 ,[7.84174125e-02, 4.75600813e-01]
 ,[7.41456469e-02, 4.95400926e-01]
 ,[6.90602116e-02, 5.05080981e-01]
 ,[6.80431245e-02, 5.21801077e-01]
 ,[6.29576892e-02, 5.31481132e-01]
 ,[5.78722539e-02, 5.48934565e-01]
 ,[5.48209927e-02, 5.60814633e-01]
 ,[4.85150529e-02, 5.83443334e-01]
 ,[4.61757526e-02, 6.00414860e-01]
 ,[4.52603743e-02, 6.01357722e-01]
 ,[4.41415785e-02, 6.18224485e-01]
 ,[3.91578519e-02, 6.27757873e-01]
 ,[3.67168430e-02, 6.43597963e-01]
 ,[3.60048820e-02, 6.47557986e-01]
 ,[3.06143206e-02, 6.69998114e-01]
 ,[3.06143206e-02, 6.69998114e-01]
 ,[3.05126119e-02, 6.84518197e-01]
 ,[2.81733116e-02, 6.85838205e-01]
 ,[2.51220504e-02, 7.06958325e-01]
 ,[2.51220504e-02, 7.06958325e-01]
 ,[2.26810415e-02, 7.22798416e-01]
 ,[2.26810415e-02, 7.22798416e-01]
 ,[2.02400325e-02, 7.49198567e-01]
 ,[2.02400325e-02, 7.49198567e-01]
 ,[2.01383238e-02, 7.66358665e-01]
 ,[1.84092758e-02, 7.68998680e-01]
 ,[1.76973149e-02, 7.82198755e-01]
 ,[1.65785191e-02, 7.88798793e-01]
 ,[1.59682669e-02, 8.05958891e-01]
 ,[1.59682669e-02, 8.05958891e-01]
 ,[1.34255492e-02, 8.28399019e-01]
 ,[1.23067535e-02, 8.34999057e-01]
 ,[1.15947925e-02, 8.48199133e-01]
 ,[1.15947925e-02, 8.48199133e-01]
 ,[9.15378356e-03, 8.74599283e-01]
 ,[8.54353133e-03, 8.81199321e-01]
 ,[6.71277461e-03, 9.00999434e-01]
 ,[6.71277461e-03, 9.00999434e-01]
 ,[4.88201790e-03, 9.20799547e-01]
 ,[4.27176566e-03, 9.27399585e-01]
 ,[2.44100895e-03, 9.73599849e-01]
 ,[2.44100895e-03, 9.73599849e-01]
 ,[0.00000000e+00, 1.00000000e+00]
 ,[0.00000000e+00, 1.00000000e+00]]
)
# print(best_pare_to_front)
Fcapacity_max_best =  115.87619047619049
Fcapacity_min_best =  40.11904761904762
Fdistance_max_best =  103150.0
Fdistance_min_best =  53990.0

In [ ]:
# problem = Problem(num_of_variables = number_of_blocks, objectives = [fdistance, fcapacity], variables_range = number_of_shelters)
# evo = Evolution(problem, num_of_generations = 500, mutation_param =  0.13630410571719337, tournament_prob = 1.0, num_of_individuals = 170,  num_of_tour_particips = 10, use_threshold_flag = False)
# Front ,_ = evo.evolve()
# Fcapacity = []    
# Fdistance = []
# pare_to = [[]]
# counter = 0
# plt.xlabel('Fcapacity', fontsize= 15)
# plt.ylabel('Fdistance', fontsize= 15)
# for i in Front:
#     Fcapacity.append(i.objectives[1])
#     Fdistance.append(i.objectives[0])
#     pare_to.append([i.objectives[0],i.objectives[1]])
#     path = f'map_animation_{counter}.html'
#     map_animation(filename = path,individual = i.features )
#     counter += 1


# plt.scatter(Fcapacity, Fdistance)
# plt.grid(True)
# file_path = os.path.join(f'pareToFront.png')
# plt.savefig(file_path, format="png")
# plt.close()

# for i in range(len(Front)):
#     if i == 10 : 
#         print("Fcapcity is most important",Front[i].features)
#     if i == len(Front) // 2 :
#         print("both are important",Front[i].features)
#     if i == len(Front) - 10:
#         print("Fdistance is most important",Front[i].features)       
#     path = f'map_animation_{i}.html'
#     convert_html_to_png(path =  path, pare_to_number= i )   
# make_animation(Front = Front)


        


In [9]:
# def metric_of_spacing(Front):
#         number_of_individuals_in_pare_to = Front.shape[0]
#         Distance = np.zeros(number_of_individuals_in_pare_to)

#         for i in range(number_of_individuals_in_pare_to):
#             #copy each individual objectives values for the number of individuals in pare to front in D1
#             Objectives = np.tile(Front[i, :], (number_of_individuals_in_pare_to, 1)) 
#             #the difference between the i th indiuval objectives values and each individual objectives values will be savea at D2  
#             Diff = Objectives - Front
#             # the manhattan distance will be calculated for each row of D2
#             Manhattan_Distances = np.sum(np.abs(Diff), axis=1)
#             Manhattan_Distances = np.delete(Manhattan_Distances, i)
#             # nearest distance
#             Distance[i] = np.min(Manhattan_Distances)
#         return np.std(Distance)


# data_first_df = pd.DataFrame(columns=['Run_number', 'number_of_generations', 'hypervolume','spread','convergence','Execution_time', 'Number_of_solutions', 'Fdistance_best_value', 'Fcapacity_best_value','Fdistance_worst_value', 'Fcapacity_worst_value', 'Fdistance_average_value', 'Fcapacity_average_value'])
# data_second_df = pd.DataFrame(columns=['Run_number', 'number_of_generations','hypervolume','spread','convergence','Execution_time', 'Number_of_solutions', 'Fdistance_best_value', 'Fcapacity_best_value','Fdistance_worst_value', 'Fcapacity_worst_value', 'Fdistance_average_value', 'Fcapacity_average_value'])

# for i in range(10):
#     problem = Problem(num_of_variables = number_of_blocks, objectives = [fdistance, fcapacity], variables_range = number_of_shelters)
#     evo = Evolution(problem, num_of_generations = 500, mutation_param = 0.01, tournament_prob = 0.9, num_of_individuals = 100, num_of_tour_particips = 2, use_threshold_flag = True)
#     start_time = time.time()
#     Front, number_of_genrations = evo.evolve()
#     end_time = time.time()
#     Execution_time = end_time - start_time
#     converge = 0
#     hypervolume = 0
#     spread = 0
#     Fcapacity = []    
#     Fdistance = []
#     pare_to_front = [[] for _ in range(len(Front))]
#     for j in Front:
#         Fcapacity.append(j.objectives[1])
#         Fdistance.append(j.objectives[0])
#         # map_animation(Fronts[i][j].features,  filename)
#     Fcapacity_max = max(Fcapacity)
#     Fcapacity_min = min(Fcapacity)
#     Fdistance_max = max(Fdistance)
#     Fdistance_min = min(Fdistance)
#     Fdistance_avg = sum(Fdistance) / len(Fdistance)
#     Fcapacity_avg = sum(Fcapacity) / len(Fcapacity)
#     for m in range (len(Fcapacity)):
#         pare_to_front[m] = [(Fdistance[m] - Fdistance_min_best)/ (Fdistance_max_best - Fdistance_min_best), (Fcapacity[m] - Fcapacity_min_best)/ (Fcapacity_max_best - Fcapacity_min_best)]
    
#     pare_to_front = np.array(pare_to_front)
#     igd = IGD(best_pare_to_front)
#     converge = igd(pare_to_front)   
    
#     for m in range (len(Fcapacity)):    
#         Fdistance[m] = (Fdistance[m] - Fdistance_min) / (Fdistance_max - Fdistance_min)
#         Fcapacity[m] = (Fcapacity[m] - Fcapacity_min) / (Fcapacity_max - Fcapacity_min)
#         pare_to_front[m] = [Fdistance[m], Fcapacity[m]]
    
#     reference_point  = np.array([max(Fdistance) + 0.000001, max(Fcapacity) + 0.000001])
#     pare_to_front = np.array(pare_to_front)
#     hypervolume_indicator = Hypervolume(ref_point = reference_point)
#     hypervolume = hypervolume_indicator.do(pare_to_front)

#     spread = metric_of_spacing(pare_to_front)
#     data = {
#         'Run_number': i + 1,
#         'number_of_generations': number_of_genrations,
#         'hypervolume' : hypervolume,
#         'spread' : spread,
#         'convergence': converge,
#         'Execution_time': Execution_time,
#         'Number_of_solutions': len(Front),
#         'Fdistance_worst_value': Fdistance_max,
#         'Fcapacity_worst_value': Fcapacity_max,
#         'Fdistance_average_value': Fdistance_avg,
#         'Fcapacity_average_value': Fcapacity_avg,
#         'Fdistance_best_value' : Fdistance_min, 
#         'Fcapacity_best_value' : Fcapacity_min,
#     }
#     data_first_df = pd.concat([data_first_df, pd.DataFrame([data])], ignore_index=True)
       
# file_path = os.path.join('results_for_first.csv')
# data_first_df.to_csv(file_path, index=False)

# for i in range(10):
#     problem = Problem(num_of_variables = number_of_blocks, objectives = [fdistance, fcapacity], variables_range = number_of_shelters)
#     evo = Evolution(problem, num_of_generations = 500, mutation_param =  0.13630410571719337, tournament_prob = 1.0, num_of_individuals = 170, num_of_tour_particips = 10, use_threshold_flag = True)
#     start_time = time.time()
#     Front, number_of_genrations = evo.evolve()
#     end_time = time.time()
#     Execution_time = end_time - start_time
#     converge = 0
#     hypervolume = 0
#     spread = 0
#     Fcapacity = []    
#     Fdistance = []
#     pare_to_front = [[] for _ in range(len(Front))]
#     for j in Front:
#         Fcapacity.append(j.objectives[1])
#         Fdistance.append(j.objectives[0])

#     Fcapacity_max = max(Fcapacity)
#     Fcapacity_min = min(Fcapacity)
#     Fdistance_max = max(Fdistance)
#     Fdistance_min = min(Fdistance)
#     Fdistance_avg = sum(Fdistance) / len(Fdistance)
#     Fcapacity_avg = sum(Fcapacity) / len(Fcapacity)
   
#     for m in range (len(Fcapacity)):
#         pare_to_front[m] = [(Fdistance[m] - Fdistance_min_best)/ (Fdistance_max_best - Fdistance_min_best), (Fcapacity[m] - Fcapacity_min_best)/ (Fcapacity_max_best - Fcapacity_min_best)]
    
#     pare_to_front = np.array(pare_to_front)
#     igd = IGD(best_pare_to_front)
#     converge = igd(pare_to_front)   
    
#     for m in range (len(Fcapacity)):    
#         Fdistance[m] = (Fdistance[m] - Fdistance_min) / (Fdistance_max - Fdistance_min)
#         Fcapacity[m] = (Fcapacity[m] - Fcapacity_min) / (Fcapacity_max - Fcapacity_min)
#         pare_to_front[m] = [Fdistance[m], Fcapacity[m]]
    
#     reference_point  = np.array([max(Fdistance) + 0.000001, max(Fcapacity) + 0.000001])
#     pare_to_front = np.array(pare_to_front)
#     hypervolume_indicator = Hypervolume(ref_point = reference_point)
#     hypervolume = hypervolume_indicator.do(pare_to_front)

#     spread = metric_of_spacing(pare_to_front)
#     data = {
#         'Run_number': i + 1,
#         'number_of_generations': number_of_genrations,
#         'hypervolume' : hypervolume,
#         'spread' : spread,
#         'convergence': converge,
#         'Execution_time': Execution_time,
#         'Number_of_solutions': len(Front),
#         'Fdistance_worst_value': Fdistance_max,
#         'Fcapacity_worst_value': Fcapacity_max,
#         'Fdistance_average_value': Fdistance_avg,
#         'Fcapacity_average_value': Fcapacity_avg,
#         'Fdistance_best_value' : Fdistance_min, 
#         'Fcapacity_best_value' : Fcapacity_min,
#     }
#     data_second_df = pd.concat([data_second_df, pd.DataFrame([data])], ignore_index=True)
       
# file_path = os.path.join('results_for_second.csv')
# data_second_df.to_csv(file_path, index=False)  

 50%|█████     | 250/500 [00:48<00:48,  5.11it/s]
C:\Users\mahdi\AppData\Local\Temp\ipykernel_21788\3187757829.py:77: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_first_df = pd.concat([data_first_df, pd.DataFrame([data])], ignore_index=True)
 62%|██████▏   | 310/500 [03:14<01:58,  1.60it/s]
C:\Users\mahdi\AppData\Local\Temp\ipykernel_21788\3187757829.py:139: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_second_df = pd.concat([data_second_df, pd.DataFrame([data])], ignore_index=True)
 

In [44]:
# def metric_of_spacing(Front):
#         number_of_individuals_in_pare_to = Front.shape[0]
#         Distance = np.zeros(number_of_individuals_in_pare_to)

#         for i in range(number_of_individuals_in_pare_to):
#             #copy each individual objectives values for the number of individuals in pare to front in D1
#             Objectives = np.tile(Front[i, :], (number_of_individuals_in_pare_to, 1)) 
#             #the difference between the i th indiuval objectives values and each individual objectives values will be savea at D2  
#             Diff = Objectives - Front
#             # the manhattan distance will be calculated for each row of D2
#             Manhattan_Distances = np.sum(np.abs(Diff), axis=1)
#             Manhattan_Distances = np.delete(Manhattan_Distances, i)
#             # nearest distance
#             Distance[i] = np.min(Manhattan_Distances)
#         return np.std(Distance)
# problem = Problem(num_of_variables = number_of_blocks, objectives = [fdistance, fcapacity], variables_range = number_of_shelters)
# evo = Evolution(problem, num_of_generations = 500, mutation_param = 0.01, tournament_prob = 0.9, num_of_individuals = 100, num_of_tour_particips = 2, use_threshold_flag = False)
# Fronts = evo.evolve()
# problem = Problem(num_of_variables = number_of_blocks, objectives = [fdistance, fcapacity], variables_range = number_of_shelters)
# evo = Evolution(problem, num_of_generations = 500, mutation_param =  0.13630410571719337, tournament_prob = 1.0, num_of_individuals = 170,  num_of_tour_particips = 10, use_threshold_flag = False)
# Fronts2 = evo.evolve()
# IGDs = []
# HVs = []
# spreads = []
# IGDs2 = []
# HVs2 = []
# spreads2 = []
# for i in range(len(Fronts)):  
#     Fcapacity = []    
#     Fdistance = []
#     Fcapacity2 = []    
#     Fdistance2 = []
#     pare_to_front = [[] for _ in range(len(Fronts[i]))]
#     pare_to_front2 = [[] for _ in range(len(Fronts2[i]))]
#     plt.xlabel('Fcapacity', fontsize= 15)
#     plt.ylabel('Fdistance', fontsize= 15)
#     plt.title(f'generation_{i}')
    
#     for j in range(len(Fronts[i])):
#         filename = f"map_animation_1_{i}_Pare_to_{j}.html"
#         Fcapacity.append(Fronts[i][j].objectives[1])
#         Fdistance.append(Fronts[i][j].objectives[0])
        
#     Fcapacity_max = max(Fcapacity)
#     Fcapacity_min = min(Fcapacity)
#     Fdistance_max = max(Fdistance)
#     Fdistance_min = min(Fdistance)
    
#     for m in range (len(Fcapacity)):
#         pare_to_front[m] = [(Fdistance[m] - Fdistance_min_best)/ (Fdistance_max_best - Fdistance_min_best), (Fcapacity[m] - Fcapacity_min_best)/ (Fcapacity_max_best - Fcapacity_min_best)]
    
#     pare_to_front = np.array(pare_to_front)
#     igd = IGD(best_pare_to_front)
#     converge = igd(pare_to_front)
#     IGDs.append(converge)   
    
#     for m in range (len(Fcapacity)):    
#         Fdistance[m] = (Fdistance[m] - Fdistance_min) / (Fdistance_max - Fdistance_min)
#         Fcapacity[m] = (Fcapacity[m] - Fcapacity_min) / (Fcapacity_max - Fcapacity_min)
#         pare_to_front[m] = [Fdistance[m], Fcapacity[m]]
    
#     reference_point  = np.array([max(Fdistance) + 0.000001, max(Fcapacity) + 0.000001])
#     pare_to_front = np.array(pare_to_front)
#     hypervolume_indicator = Hypervolume(ref_point = reference_point)
#     hv = hypervolume_indicator.do(pare_to_front)
#     HVs.append(hv)

#     sp = metric_of_spacing(pare_to_front)
#     spreads.append(sp)

#     for k in range(len(Fronts2[i])):
#         filename = f"map_animation_2_{i}_Pare_to_{k}.html"
#         Fcapacity2.append(Fronts2[i][k].objectives[1])
#         Fdistance2.append(Fronts2[i][k].objectives[0])

#     Fcapacity_max = max(Fcapacity2)
#     Fcapacity_min = min(Fcapacity2)
#     Fdistance_max = max(Fdistance2)
#     Fdistance_min = min(Fdistance2)
    
#     for m in range (len(Fcapacity2)):
#         pare_to_front2[m] = [(Fdistance2[m] - Fdistance_min_best)/ (Fdistance_max_best - Fdistance_min_best), (Fcapacity2[m] - Fcapacity_min_best)/ (Fcapacity_max_best - Fcapacity_min_best)]
    
#     pare_to_front2 = np.array(pare_to_front2)
#     igd = IGD(best_pare_to_front)
#     converge = igd(pare_to_front2)
#     IGDs2.append(converge)   
    
#     for m in range (len(Fcapacity2)):
#         Fdistance2[m] = (Fdistance2[m] - Fdistance_min) / (Fdistance_max - Fdistance_min)
#         Fcapacity2[m] = (Fcapacity2[m] - Fcapacity_min) / (Fcapacity_max - Fcapacity_min)
#         pare_to_front2[m] = [(Fdistance2[m] - Fdistance_min) / (Fdistance_max - Fdistance_min), (Fcapacity2[m] - Fcapacity_min) / (Fcapacity_max - Fcapacity_min)]    
    
#     reference_point  = np.array([max(Fdistance2) + 0.000001, max(Fcapacity2) + 0.000001])

#     hypervolume_indicator = Hypervolume(ref_point = reference_point)
#     hv = hypervolume_indicator.do(pare_to_front2)
#     HVs2.append(hv)
    
#     sp = metric_of_spacing(pare_to_front2)
#     spreads2.append(sp)
    
#     plt.scatter(Fcapacity, Fdistance, label = 'without tunning', color = 'blue')
#     plt.scatter(Fcapacity2, Fdistance2, label = 'with tunning', color = 'red')
#     plt.legend()
#     plt.grid(True)
#     file_path = os.path.join(f'pareToFront_of_generation_{i}.png')
#     plt.savefig(file_path, format="png")
#     plt.close()



# generations = [i for i in range(500) ]
# make_animation2()
# plt.xlabel('generations', fontsize= 15)
# plt.ylabel('hypervoulme', fontsize= 15)
# plt.title('HYPERVOLUME')
# plt.scatter(generations, HVs, label = 'without tunning', color = 'blue')
# plt.scatter( generations, HVs2,label = 'with tunning', color = 'red')
# plt.legend()
# plt.grid(True)
# file_path = os.path.join('hypervolume.png')
# plt.savefig(file_path, format="png")
# plt.close()

# plt.xlabel('generations', fontsize= 15)
# plt.ylabel('convergence', fontsize= 15)
# plt.title('CONVERGENVCE')
# plt.scatter(generations, IGDs, label = 'without tunning', color = 'blue')
# plt.scatter(generations, IGDs2, label = 'with tunning', color = 'red')
# plt.legend()
# plt.grid(True)
# file_path = os.path.join('convergence.png')
# plt.savefig(file_path, format="png")
# plt.close()

# plt.xlabel('generations', fontsize= 15)
# plt.ylabel('spread', fontsize= 15)
# plt.title('SPREAD')
# plt.scatter(generations, spreads, label = 'without tunning', color = 'blue')
# plt.scatter(generations, spreads2, label = 'with tunning', color = 'red')
# plt.legend()
# plt.grid(True)
# file_path = os.path.join('spreads.png')
# plt.savefig(file_path, format="png")
# plt.close()
 

100%|██████████| 500/500 [02:48<00:00,  2.96it/s]
C:\Users\mahdi\AppData\Local\Temp\ipykernel_8788\2262357319.py:138: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(filename))
C:\Users\mahdi\AppData\Local\Temp\ipykernel_8788\2262357319.py:139: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  print(f"Image {filename} shape: {imageio.imread(filename).shape}")


Image pareToFront_of_generation_0.png shape: (480, 640, 4)
Image pareToFront_of_generation_1.png shape: (480, 640, 4)
Image pareToFront_of_generation_2.png shape: (480, 640, 4)
Image pareToFront_of_generation_3.png shape: (480, 640, 4)
Image pareToFront_of_generation_4.png shape: (480, 640, 4)
Image pareToFront_of_generation_5.png shape: (480, 640, 4)
Image pareToFront_of_generation_6.png shape: (480, 640, 4)
Image pareToFront_of_generation_7.png shape: (480, 640, 4)
Image pareToFront_of_generation_8.png shape: (480, 640, 4)
Image pareToFront_of_generation_9.png shape: (480, 640, 4)
Image pareToFront_of_generation_10.png shape: (480, 640, 4)
Image pareToFront_of_generation_11.png shape: (480, 640, 4)
Image pareToFront_of_generation_12.png shape: (480, 640, 4)
Image pareToFront_of_generation_13.png shape: (480, 640, 4)
Image pareToFront_of_generation_14.png shape: (480, 640, 4)
Image pareToFront_of_generation_15.png shape: (480, 640, 4)
Image pareToFront_of_generation_16.png shape: (480

Tuning the NSGA-II parameters using GA

metric_of_spacing source code in MATLAB from : https://www.mathworks.com/matlabcentral/fileexchange/125980-c-index-spacing-and-hypervolume


In [17]:

class chromosome(object):

    def __init__(self):
        self.features = []
        self.fitness = None
        self.hypervolume = None
        self.spread = None
        self.convergence = None
        self.Execution_time = None
        self.Number_of_solutions = None
        self.Fdistance_worst_value = None
        self.Fcapacity_worst_value = None
        self.Fdistance_average_value = None
        self.Fcapacity_average_value = None
        self.Fdistance_best_value = None
        self.Fcapacity_best_value = None

    def __eq__(self, other):
        if isinstance(self, other.__class__):
            return self.features == other.features
        return False

    

class Pop:

    def __init__(self):
        self.population = []

    def __len__(self):
        return len(self.population)

    def __iter__(self):
        return self.population.__iter__()

    def extend(self, new_individuals):
        self.population.extend(new_individuals)

    def append(self, new_individual):
        self.population.append(new_individual)
    



class GAUtils:

    def __init__(self, num_of_variables, ranges ,num_of_individuals = 100,
                 num_of_tour_particips = 2, tournament_prob = 0.9, crossover_param = 2, mutation_param = 0.01, weight_of_hypervolume = (1 / 3), weight_of_spread = (1 / 3), weight_of_convergence = (1/3)):

        self.hypervoulme = []
        self.spread = []
        self.convergence = []
        self.num_of_individuals = num_of_individuals
        self.ranges = ranges
        self.num_of_tour_particips = num_of_tour_particips
        self.tournament_prob = tournament_prob
        self.crossover_param = crossover_param
        self.mutation_param = mutation_param
        self.num_of_variables = num_of_variables
        self.weight_of_hypervolume = weight_of_hypervolume
        self.weight_of_spread = weight_of_spread
        self.weight_of_convergence = weight_of_convergence
       

    def individual_generator(self):
        individual = chromosome()
        value = 0
        for _ in range(int(self.num_of_variables)):
                #order of parametes :size of population, tournament size, mutation probability, tournament selection probability
                while value == 0:
                    value = random.random()
                individual.features.append(value)

        return individual
    
    def check_range(self, number_of_feature):
        if  ((self.ranges[number_of_feature][0] >= 0) and (self.ranges[number_of_feature][1] <= 1)) :
            return True
        return False

    def create_initial_population(self):
        population = Pop()
        for _ in range(self.num_of_individuals):
            individual = self.individual_generator()
            individual = self.calculate_fitness(individual = individual)
            population.append(individual)
        return population
    
    def scale_change(self, number, number_of_feature):
        if self.check_range(number_of_feature = number_of_feature):
            return (number * (self.ranges[number_of_feature][1] - self.ranges[number_of_feature][0])) +  self.ranges[number_of_feature][0] 
        
        return int((number * (self.ranges[number_of_feature][1] - self.ranges[number_of_feature][0]))) +  self.ranges[number_of_feature][0]
    
    def metric_of_spacing(self, Front):
        number_of_individuals_in_pare_to = Front.shape[0]
        Distance = np.zeros(number_of_individuals_in_pare_to)

        for i in range(number_of_individuals_in_pare_to):
            #copy each individual objectives values for the number of individuals in pare to front in D1
            Objectives = np.tile(Front[i, :], (number_of_individuals_in_pare_to, 1)) 
            #the difference between the i th indiuval objectives values and each individual objectives values will be savea at D2  
            Diff = Objectives - Front
            # the manhattan distance will be calculated for each row of D2
            Manhattan_Distances = np.sum(np.abs(Diff), axis=1)
            Manhattan_Distances = np.delete(Manhattan_Distances, i)
            # nearest distance
            Distance[i] = np.min(Manhattan_Distances)
        return np.std(Distance)

    def metrics_calculator(self, Front, indiviual , execution_time ):
    
      Fcapacity_Pare_To = []
      Fdistance_Pare_To = []
      PareToFront = [[] for _ in range(len(Front))]
      PareToFront2 = [[] for _ in range(len(Front))]   
      for j in Front:
          Fcapacity_Pare_To.append(j.objectives[1])
          Fdistance_Pare_To.append(j.objectives[0])
   
    
      Fcapacity_max = max(Fcapacity_Pare_To)
      Fcapacity_min = min(Fcapacity_Pare_To)
      Fdistance_max = max(Fdistance_Pare_To)
      Fdistance_min = min(Fdistance_Pare_To)
      Fdistance_avg = sum(Fdistance_Pare_To) / len(Fdistance_Pare_To)
      Fcapacity_avg = sum(Fcapacity_Pare_To) / len(Fcapacity_Pare_To)
    
      for i in range (len(Fcapacity_Pare_To)):
        PareToFront2[i] = [(Fdistance_Pare_To[i] - Fdistance_min_best)/ (Fdistance_max_best - Fdistance_min_best), (Fcapacity_Pare_To[i] - Fcapacity_min_best)/ (Fcapacity_max_best - Fcapacity_min_best)]
        PareToFront[i] = [(Fdistance_Pare_To[i] - Fdistance_min) / (Fdistance_max - Fdistance_min), (Fcapacity_Pare_To[i] - Fcapacity_min) / (Fcapacity_max - Fcapacity_min)]
    
      reference_point  = np.array([max(Fdistance_Pare_To) + 0.000001, max(Fcapacity_Pare_To) + 0.000001])
      PareToFront = np.array(PareToFront)
      PareToFront2 = np.array(PareToFront2)  
      #calculate convergence using inverted generational distance
      igd = IGD(best_pare_to_front)
      converge = igd(PareToFront2)
      self.convergence.append(converge)
      scale = max(self.convergence) - min(self.convergence)
      if scale == 0 :
          scale = 1     
      converge = (converge - min(self.convergence)) / scale
          
      #calculate hypervolume
      hypervolume_indicator = Hypervolume(ref_point = reference_point)
      hv = hypervolume_indicator.do(PareToFront)
      self.hypervoulme.append(hv)  
      scale = max(self.hypervoulme) - min(self.hypervoulme)
      if scale == 0 :
          scale = 1    
      hv = (hv - min(self.hypervoulme)) / scale

      # calculate spacing
      sp = self.metric_of_spacing(PareToFront)
      self.spread.append(sp)
      scale = max(self.spread) - min(self.spread)
      if scale == 0 :
          scale = 1      
      sp = (sp - min(self.spread)) / scale

      
      fitness = (self.weight_of_hypervolume * hv) - (self.weight_of_spread * sp) - (self.weight_of_convergence * converge) 
    
      #save some data
      indiviual.fitness = fitness
      indiviual.hypervolume = hv
      indiviual.spread = sp
      indiviual.convergence = converge
      indiviual.Execution_time =  execution_time
      indiviual.Number_of_solutions = len(Front)
      indiviual.Fdistance_worst_value = Fdistance_max
      indiviual.Fcapacity_worst_value =  Fcapacity_max
      indiviual.Fdistance_average_value = Fdistance_avg
      indiviual.Fcapacity_average_value =  Fcapacity_avg
      indiviual.Fdistance_best_value = Fdistance_min 
      indiviual.Fcapacity_best_value = Fcapacity_min
    
      return indiviual

    def calculate_fitness(self, individual):
        problem = Problem(num_of_variables = number_of_blocks, objectives = [fdistance, fcapacity], variables_range = number_of_shelters)
        evo = Evolution(problem, num_of_generations = 500, num_of_individuals = self.scale_change(number = individual.features[0], number_of_feature = 0) , num_of_tour_particips = self.scale_change(number = individual.features[1], number_of_feature = 1) ,mutation_param =  self.scale_change(number = individual.features[2], number_of_feature = 2), tournament_prob = self.scale_change(number = individual.features[3], number_of_feature = 3))
        start_time = time.time()
        Front = evo.evolve()
        end_time = time.time()
        individual = self.metrics_calculator(Front, individual, end_time - start_time)
        return individual 
    
    def create_children(self, population):
        children = []
        while len(children) < (len(population) - 1):
            parent1 = self.__tournament(population)
            parent2 = parent1
            while parent1 == parent2:
                parent2 = self.__tournament(population)
            child1, child2 = self.__crossover(parent1, parent2)
            if self.__choose_with_prob(self.mutation_param):
                child1 = self.__mutate(child1)
            if self.__choose_with_prob(self.mutation_param):
                child2 = self.__mutate(child2)
            child1 = self.calculate_fitness(child1)
            child2 = self.calculate_fitness(child2)
            children.append(child1)
            children.append(child2)
        return children
        
    def __crossover(self, parent1, parent2):
        child1 = self.individual_generator()
        child2 = self.individual_generator()
        num_of_features = len(child1.features)
        genes_indexes = range(num_of_features)
        for i in genes_indexes:
            beta = self.__get_beta()
            x1 = (parent1.features[i] + parent2.features[i]) / 2
            x2 = abs((parent1.features[i] - parent2.features[i]) / 2)
            child1.features[i] = np.clip(x1 + beta * x2, 0, 1)
            child2.features[i] = np.clip(x1 - beta * x2, 0, 1)
        return child1, child2

    def __get_beta(self):
        u = random.random()
        if u <= 0.5:
            return (2 * u) ** (1 / (self.crossover_param + 1))
        return (2 * (1 - u)) ** (-1 / (self.crossover_param + 1))  
      
    def __mutate(self, child):
        number_of_features_to_mutate = random.randint(1, len(child.features) - 1)
        for _ in range(number_of_features_to_mutate):
            feature_to_change = random.randint(0, len(child.features) - 1) 
        #order of parametes : size of population, tournament size, mutation probability, tournament selection probability   
            mean = 0
            std_dev = 0.15
            child.features[feature_to_change] += np.random.normal(mean, std_dev)  
            child.features[feature_to_change] = np.clip(child.features[feature_to_change], 0, 1)    
        return child

    def __tournament(self, population):
        participants = random.sample(population.population, self.num_of_tour_particips)
        best = None
        for participant in participants:
            if best is None or (
                    participant.fitness > best.fitness and self.__choose_with_prob(self.tournament_prob)):
                best = participant
        return best

    def __choose_with_prob(self, prob):
        if random.random() <= prob:
            return True
        return False

class Evo:

    def __init__(self, num_of_variables = 4, ranges = [[30,150],[2,10],[0.01,0.15],[0.7,1]], num_of_generations = 100, num_of_individuals = 20, num_of_tour_particips = 2,
                 tournament_prob = 0.8, crossover_param = 2, mutation_param = 0.01 ,  weight_of_hypervolume = 1/3, weight_of_spread = 1/3, weight_of_convergence = 1/3):
        self.utils = GAUtils(num_of_variables = num_of_variables,  ranges = ranges ,num_of_individuals = num_of_individuals,
                 num_of_tour_particips = num_of_tour_particips, tournament_prob = tournament_prob, crossover_param = crossover_param, mutation_param = mutation_param, weight_of_hypervolume = weight_of_hypervolume, weight_of_spread = weight_of_spread, weight_of_convergence = weight_of_convergence)
        self.population = None
        # self.ranges = ranges
        self.num_of_generations = num_of_generations
        self.num_of_individuals = num_of_individuals
        # self.num_of_variables = num_of_variables

    def save_data(self,individual, parameters_df):
        data = {
        'size_of_population': self.utils.scale_change(number = individual.features[0], number_of_feature = 0),
        'tournament_size': self.utils.scale_change(number = individual.features[1], number_of_feature = 1),
        'mutation_probability': self.utils.scale_change(number = individual.features[2], number_of_feature = 2),
        'tournament_selection_probability': self.utils.scale_change(number = individual.features[3], number_of_feature = 3),
        'hypervolume': individual.hypervolume,
        'spread': individual.spread,
        'convergence': individual.convergence,
        'fitness': individual.fitness,
        'Execution_time': individual.Execution_time,
        'Number_of_solutions': individual.Number_of_solutions,
        'Fdistance_worst_value': individual.Fdistance_worst_value,
        'Fcapacity_worst_value':  individual.Fcapacity_worst_value,
        'Fdistance_average_value': individual.Fdistance_average_value,
        'Fcapacity_average_value': individual.Fcapacity_average_value,
        'Fdistance_best_value' : individual.Fdistance_best_value, 
        'Fcapacity_best_value' : individual.Fcapacity_best_value,
        }
        parameters_df = pd.concat([parameters_df, pd.DataFrame([data])], ignore_index=True)
        return parameters_df

    def evolve(self):
        parameters_df = pd.DataFrame(columns=['size_of_population', 'tournament_size', 'mutation_probability', 'tournament_selection_probability','hypervolume', 'spread', 'convergence', 'fitness' ,'Execution_time', 'Number_of_solutions','Fdistance_best_value', 'Fcapacity_best_value','Fdistance_worst_value', 'Fcapacity_worst_value', 'Fdistance_average_value', 'Fcapacity_average_value'])
        self.population = self.utils.create_initial_population()
        returned_individual = None
        for _ in range(self.num_of_generations):
            children = self.utils.create_children(self.population) #it will calculate the fitness of each children inside the function. 
            self.population.extend(children)
            for individual in self.population :
                parameters_df = self.save_data(individual = individual, parameters_df = parameters_df)
            new_population = Pop()
            self.population.population.sort(key=lambda individual: individual.fitness, reverse=True)
            new_population.extend(self.population.population[0:self.num_of_individuals])        
            self.population = new_population
            returned_individual = self.population.population[0] 
        file_path = os.path.join('results.csv')
        parameters_df.to_csv(file_path, index=False)
        return  returned_individual
    

 56%|█████▌    | 280/500 [01:32<01:12,  3.03it/s]
C:\Users\mahdi\AppData\Local\Temp\ipykernel_24336\1185767313.py:285: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  parameters_df = pd.concat([parameters_df, pd.DataFrame([data])], ignore_index=True)
 74%|███████▍  | 370/500 [01:38<00:34,  3.75it/s]

number of generations:  170
number of tournament participant:  10
mutation prob:  0.13630410571719337
tornament prob:  1.0


In [25]:
evo = Evo(num_of_variables = 4, ranges = [[50,170],[2,10],[0.01,0.15],[0.7,1]], num_of_generations = 20 , mutation_param = 0.1, num_of_tour_particips = 4, tournament_prob = 0.9, num_of_individuals = 70)
parameters = evo.evolve()
print('size of population: ', int(parameters.features[0] * 120 )+ 50)
print('number of tournament participant: ',int(parameters.features[1] * 8) + 2)
print('mutation prob: ', parameters.features[2] * 0.14 +  0.01)
print('tornament prob: ',parameters.features[3] * 0.3 +  0.7)

[32, 21, 25, 9, 8, 29, 3, 17, 41, 13, 25, 33, 18, 12, 19, 12, 31, 38, 12, 38, 14, 20, 32, 29, 13, 2, 1, 8, 29, 31, 17, 4, 37, 29, 10, 41, 42, 38, 1, 8, 31, 39, 30, 16]
[18, 11, 12, 41, 42, 20, 42, 18, 23, 1, 42, 31, 23, 28, 30, 31, 12, 28, 3, 29, 36, 37, 36, 40, 35, 14, 31, 25, 3, 23, 5, 32, 7, 31, 10, 38, 1, 11, 19, 13, 16, 28, 21, 38]
[22, 12, 34, 9, 12, 12, 34, 31, 12, 41, 25, 25, 6, 15, 16, 28, 7, 9, 16, 13, 3, 10, 37, 13, 32, 25, 11, 39, 1, 1, 28, 42, 16, 18, 29, 28, 26, 3, 40, 5, 10, 13, 5, 35]
[42, 40, 12, 28, 17, 25, 13, 3, 13, 11, 2, 42, 36, 38, 15, 42, 25, 23, 2, 31, 19, 14, 39, 31, 39, 17, 14, 29, 8, 9, 13, 23, 5, 40, 20, 29, 33, 35, 4, 18, 32, 2, 15, 10]
[31, 41, 25, 28, 36, 32, 28, 4, 26, 21, 16, 34, 41, 21, 13, 35, 33, 9, 5, 22, 2, 11, 40, 38, 19, 34, 8, 33, 3, 37, 33, 30, 35, 39, 28, 30, 17, 42, 8, 29, 21, 18, 37, 19]
[29, 37, 35, 6, 14, 19, 16, 29, 39, 9, 34, 1, 21, 10, 2, 37, 32, 20, 12, 10, 15, 6, 16, 18, 25, 8, 16, 33, 26, 39, 42, 21, 16, 9, 5, 10, 30, 5, 40, 24, 34,

 68%|██████▊   | 340/500 [01:45<00:49,  3.22it/s]
